In [1]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm
from sort import Sort
import os

%matplotlib inline

/home/lgudino/.virtualenv/hdtf/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'tkinter'

In [6]:
!sudo apt-get install python3-tk

[sudo] contraseña para lgudino: 


In [ ]:
# Model Definition remains the same
class Net(torch.nn.Module):
    def __init__(self, sequence):
        super(Net, self).__init__()

        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights='ResNet50_Weights.DEFAULT')
        self.model.fc = nn.Identity()
        self.sequence = sequence
        
    def forward(self, x):
        return self.model(x)
    
    def get_embedding(self, bbox, frame_num, camera_id):

        name = 'vdo10.avi' if camera_id == 'c015' else 'vdo.avi'
        path = f"aic19-track1-mtmc-train/train/{self.sequence}/{camera_id}/{name}"
        cap = cv2.VideoCapture(path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        cap.release()
        frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = torch.tensor((cv2.resize(frame, (224, 224)) / 255.0).transpose(2, 0, 1), dtype=torch.float32).unsqueeze(0)
        return self.model(frame)[0]

In [ ]:
video_paths = ['aic19-track1-mtmc-train/train/S01/c001/vdo.avi', 
               'aic19-track1-mtmc-train/train/S01/c002/vdo.avi', 
               'aic19-track1-mtmc-train/train/S01/c003/vdo.avi',
               'aic19-track1-mtmc-train/train/S01/c004/vdo.avi',
               'aic19-track1-mtmc-train/train/S01/c005/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c010/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c011/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c012/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c013/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c014/vdo.avi',
               'aic19-track1-mtmc-train/train/S03/c015/vdo10.avi',
               'aic19-track1-mtmc-train/train/S04/c016/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c017/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c018/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c019/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c020/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c021/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c022/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c023/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c024/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c025/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c026/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c027/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c028/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c029/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c030/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c031/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c032/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c033/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c034/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c035/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c036/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c037/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c038/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c039/vdo.avi',
               'aic19-track1-mtmc-train/train/S04/c040/vdo.avi']

In [ ]:
def upscale_fps(input_path, output_path, input_fps=8, target_fps=10):
    """
    This function upscales the FPS of a video by interpolating frames using OpenCV and NumPy.
    
    Parameters:
    input_path (str): The path to the input video file.
    output_path (str): The path where the output video will be saved.
    input_fps (int): The original FPS of the video.
    target_fps (int): The target FPS to upscale the video to.
    """
    import cv2
    import numpy as np
    
    # Capture video from input path
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        return "Failed to open video file."
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, target_fps, (width, height))
    
    prev_frame = None
    success, frame = cap.read()
    while success:
        out.write(frame)  # Write the original frame
        
        # If there's a previous frame, generate interpolated frames
        if prev_frame is not None:
            # Generate interpolated frames
            for _ in range((target_fps // input_fps) - 1):
                interp_frame = cv2.addWeighted(prev_frame, 0.5, frame, 0.5, 0)
                out.write(interp_frame)
                
        prev_frame = frame
        success, frame = cap.read()
        
    # Release everything if job is finished
    cap.release()
    out.release()
    return "Video FPS upscaled successfully."

# upscale_fps('aic19-track1-mtmc-train/train/S03/c015/vdo.avi', 'aic19-track1-mtmc-train/train/S03/c015/vdo10.avi')

In [ ]:
def create_detections(video_path, yolo):
    """
    Given a video, create detections for each frame in the video and store bounding boxes in a text file
    """
    cap = cv2.VideoCapture(video_path)

    # Create a text file to store the bounding boxes
    f = open(f"detections/{video_path.split('/')[-2].split('.')[0]}.txt", "w")

    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Get the detections for the frame
        detections = yolo(frame, verbose=False)

        # Write the bounding boxes to the text file
        for box in detections[0].boxes:
            if box.cls == 2:
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                f.write(f"{frame_num},{x1},{y1},{x2},{y2}\n")

        frame_num += 1

    f.close()
    cap.release()

# for video in tqdm(video_paths):
#     create_detections(video, yolo)

In [ ]:
def convert_detections_to_sort_format(detections):
    """
    Convert detections to the format expected by the SORT algorithm. Stacks all the detections of the same frame into a single array.
    """
    max_frame = detections[-1][0]
    all_detections = []
    for frame_num in range(max_frame + 1):
        frame_detections = [d[1:] + [1] for d in detections if d[0] == frame_num]
        if len(frame_detections) == 0:
            # print(f"No detections found for frame {frame_num}")
            frame_detections = np.empty((0, 5))
        all_detections.append(frame_detections)
    return all_detections

def compute_similarity(embedding1, embedding2):
    """
    Compute the Euclidean distance between two embeddings.
    """
    return np.linalg.norm(embedding1 - embedding2)

# def update_global_tracks(tracked_detections, camera_id, global_tracks, siamese_nn, frame_number, similarity_threshold=9):
#     """
#     Update global tracks with new detections from a specific camera.
    
#     Parameters:
#     - tracked_detections: Detections from the current camera for a specific frame.
#     - camera_id: Identifier for the current camera.
#     - global_tracks: List of global tracks across all cameras.
#     - siamese_nn: Siamese neural network model for embedding generation.
#     - frame_number: The current frame number being processed.
#     - similarity_threshold: Threshold for considering two detections as the same object.
#     """
#     for detection in tracked_detections:
#         # Generate embedding for the current detection using siamese_nn
#         with torch.no_grad():
#             detection_embedding = siamese_nn.get_embedding(detection, frame_number, camera_id)

        
#         best_match = None
#         min_distance = float('inf')
        
#         # Attempt to find the best match for the detection in existing global tracks
#         for track in global_tracks:
#             if track['camera_id'] != camera_id:  # Only compare with tracks from other cameras
#                 for track_embedding in track['embeddings']:
#                     distance = compute_similarity(detection_embedding, track_embedding)
#                     if distance < min_distance:
#                         min_distance = distance
#                         best_match = track
        
#         # If a match is found within the similarity threshold, update the track
#         if min_distance <= similarity_threshold:
#             best_match['detections'].append(detection)
#             best_match['embeddings'].append(detection_embedding)
#             best_match['last_seen'] = frame_number

#         else:
#             # No match found, create a new global track
#             new_track = {
#                 'track_id': len(global_tracks) + 1,
#                 'detections': [detection],
#                 'embeddings': [detection_embedding],
#                 'camera_id': camera_id,
#                 'last_seen': frame_number
#             }
#             global_tracks.append(new_track)

#     return global_tracks

def update_global_tracks(tracked_detections, camera_id, global_tracks, siamese_nn, frame_number, similarity_threshold=9):
    """
    Update global tracks with new detections from a specific camera.
    
    Parameters:
    - tracked_detections: Detections from the current camera for a specific frame.
    - camera_id: Identifier for the current camera.
    - global_tracks: List of global tracks across all cameras.
    - siamese_nn: Siamese neural network model for embedding generation.
    - frame_number: The current frame number being processed.
    - similarity_threshold: Threshold for considering two detections as the same object.
    """
    for detection in tracked_detections:
        # Generate embedding for the current detection using siamese_nn
        with torch.no_grad():
            detection_embedding = siamese_nn.get_embedding(detection, frame_number, camera_id)

        best_match = None
        min_distance = float('inf')
        
        # Attempt to find the best match for the detection in existing global tracks
        for track in global_tracks:
            # No longer restrict to tracks from other cameras only
            for track_embedding in track['embeddings']:
                distance = compute_similarity(detection_embedding, track_embedding)
                if distance < min_distance:
                    min_distance = distance
                    best_match = track
        
        # If a match is found within the similarity threshold, update the track
        if min_distance <= similarity_threshold:
            # Add detection to the best matching track without duplicating camera detections
            if not any(d['camera_id'] == camera_id and d['frame_number'] == frame_number for d in best_match['detections']):
                best_match['detections'].append({'detection': detection, 'camera_id': camera_id, 'frame_number': frame_number})
                best_match['embeddings'].append(detection_embedding)
                best_match['last_seen'] = frame_number
        else:
            # No match found, create a new global track
            new_track = {
                'track_id': len(global_tracks) + 1,
                'detections': [{'detection': detection, 'camera_id': camera_id, 'frame_number': frame_number}],
                'embeddings': [detection_embedding],
                'camera_id': camera_id,  # This attribute is now less about exclusivity and more about the track's origin
                'last_seen': frame_number
            }
            global_tracks.append(new_track)

    return global_tracks

def track_across_cameras(detections, camera_trackers, siamese_net):
    global_tracks = []  # Initialize global tracks
    
    counter = 0
    for camera_id, dets in detections.items():
        dets_sort = convert_detections_to_sort_format(dets)
        for frame_number, d in tqdm(enumerate(dets_sort), total=len(dets_sort)):
            tracked_detections = camera_trackers[camera_id].update(np.array(d)).astype(int)
            # keep bbox inside frame
            tracked_detections = [[max(0, x) for x in det] for det in tracked_detections]
            global_tracks = update_global_tracks(tracked_detections, camera_id, global_tracks, siamese_net, frame_number)

            if (frame_number+1) % 150 == 0:
                counter += 1
                break
        
        if counter == 2:
            break
    # Step 3 (Optional): Handle cross-camera tracking logic
    # This could involve additional steps to refine track matching across cameras,
    # especially if tracks move from the view of one camera to another.
    return global_tracks

sequence = 'S01'
if sequence == 'S01':
    camera_ids = ['c001', 'c002', 'c003', 'c004', 'c005']
elif sequence == 'S03':
    camera_ids = ['c010', 'c011', 'c012', 'c013', 'c014', 'c015']
elif sequence == 'S04':
    camera_ids = ['c016', 'c017', 'c018', 'c019', 'c020', 'c021', 'c022', 'c023', 'c024', 'c025', 'c026', 'c027', 'c028', 'c029', 'c030', 'c031', 'c032', 'c033', 'c034', 'c035', 'c036', 'c037', 'c038', 'c039', 'c040']

siamese_net = Net(sequence).eval()
yolo = YOLO("yolov8n.pt")

# Load detections from text files
detections = {}
for camera_id in camera_ids:
    with open(f'detections/{camera_id}.txt', 'r') as f:
        lines = f.readlines()
        detections[camera_id] = [[int(x) for x in line.strip().split(',')] for line in lines]

# Initialization
camera_trackers = {camera_id: Sort(iou_threshold=0.4) for camera_id in camera_ids}  # SORT tracker for each camera
global_tracks = track_across_cameras(detections, camera_trackers, siamese_net)

In [31]:
def visualize_tracking(video_path, camera_id, global_tracks, output_path):
    """
    Visualize tracking by drawing bounding boxes and track IDs on video frames for a specific camera.

    Parameters:
    - video_path: Path to the video file for the camera.
    - camera_id: ID of the camera to visualize detections for.
    - global_tracks: List of global tracks, each containing detections from multiple cameras.
    - output_path: Path to save the output video with tracking visualization.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'MP4V'), frame_rate, (frame_width, frame_height))

    frame_number = 0
    for i in tqdm(range(0, 150)):
        ret, frame = cap.read()
        if not ret:
            break

        # Iterate through all tracks and their detections
        for track in global_tracks:
            for detection in track['detections']:
                # Check if the detection is for the current frame and camera
                if detection['camera_id'] == camera_id and detection['frame_number'] == frame_number:
                    bbox = detection['detection']
                    x, y, x2, y2 = bbox[:4]
                    cv2.rectangle(frame, (x, y), (x2, y2), (0, 127, 0), 2)
                    cv2.putText(frame, f"ID: {track['track_id']}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 127, 0), 2)

        out.write(frame)
        frame_number += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()


# Load the video
for c in range(0,2):
    video_path = video_paths[c]
    camera_id = video_path.split('/')[-2]
    output_path = f'tracking_output_{camera_id}.mp4' 
    visualize_tracking(video_path, camera_id, global_tracks, output_path)

  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [00:10<00:00, 14.79it/s]


In [ ]:
global_tracks